In [212]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using DifferentialEquations
using MeshIO

using VMRobotControl: remake
include("functions.jl")

display_transform (generic function with 1 method)

## Creating a simple mechanism for test purposes

In [213]:
mechanism = Mechanism{Float64}("TestRobot")
F0 = root_frame(mechanism)
F1 = add_frame!(mechanism; id="L1_frame")

# Define the unit vectors to make things easier
X = SVector(1., 0., 0.)
Y = SVector(0., 1., 0.)
Z = SVector(0., 0., 1.)

J1 = Prismatic(X)
add_joint!(mechanism, J1; parent="root_frame", child="L1_frame", id="J1")

"J1"

In [214]:
add_coordinate!(mechanism, FrameOrigin(F0); id="base_centre_of_mass")
add_coordinate!(mechanism, FrameOrigin(F1); id="ee_centre_of_mass")

add_component!(mechanism, PointMass(1.0, "base_centre_of_mass"); id="base_mass")
add_component!(mechanism, PointMass(1.0, "ee_centre_of_mass"); id="ee_mass")

I_mat = @SMatrix [
    0.01  0.    0.  ;
    0.    0.01  0.  ;
    0.    0.    0.01
]

add_inertia!(mechanism, F1, I_mat; id="L1_inertia")

1DOF Mechanism{Float64} "TestRobot" with 2 frames, 1 joint, 3 coordinates, 3 components

In [215]:
add_coordinate!(mechanism, JointSubspace("J1"); id="J1")
add_component!(mechanism, LinearDamper(0.5, "J1"); id="J1_damper")
add_gravity_compensation!(mechanism, VMRobotControl.DEFAULT_GRAVITY)

In [216]:
m = compile(mechanism)
kcache = Observable(new_kinematics_cache(m))

# Setup the figure
fig1 = Figure(size=(700, 350))
ls = LScene(fig1[1, 1]; show_axis=false)

# Or a sketch of it's kinematic structure
robotsketch!(ls, kcache; scale=0.3, linewidth=2.5, transparency=true)

display(fig1)

GLMakie.Screen(...)

## Introducing virtual mechanisms

In [252]:
vms = VirtualMechanismSystem("testVMS", mechanism)

obstacle_radius = 1.0
obstacle_pos = -1.5*X

add_coordinate!(vms, FrameOrigin(".robot.L1_frame"); id="ee_position")
add_coordinate!(vms, FramePoint(".robot.root_frame", obstacle_pos); id="obstacle_position")
add_coordinate!(vms, CoordDifference("ee_position", "obstacle_position"); id="obstacle_error")
add_coordinate!(vms, CoordNorm("obstacle_error"); id="obstacle_error_norm")
add_coordinate!(vms, ConstCoord(obstacle_radius); id = "relu_spring_length")
add_coordinate!(vms, CoordDifference("obstacle_error_norm", "relu_spring_length"); id="shifted_obstacle_error")

add_component!(vms, ReLUSpring(100.0, "shifted_obstacle_error", true); id="relu spring")
add_component!(vms, RectifiedDamper(100.0, "obstacle_error_norm", (0.0, 1.1*obstacle_radius), true, false); id = "relu damper")

add_component!(vms, ForceSource(SVector(-0.5),10.0, ".robot.J1"); id="force source") # Just to see the effect of the "obstacle"

"force source"

## Simulating

In [253]:
tspan = (0., 8.)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), Float64[]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), Float64[]) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan)

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

ODE   0%|█                                              |  ETA: N/A
ODE 100%|███████████████████████████████████████████████| Time: 0:00:00


## Visualizing the result

In [254]:
display(fig1)

animate_robot_odesolution(fig1, sol, kcache, "robot_animation.mp4")

LoadError: Screen not open!